In [14]:
spark

In [15]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [16]:
raw_data_path = '/home/cs229/music_data_raw/'
# See https://www.kaggle.com/c/kkbox-music-recommendation-challenge/data for data description
# members.csv
# sample_submission.csv
# song_extra_info.csv
# songs.csv
# test.csv
# train.csv

In [4]:
member_df = spark.read.load(raw_data_path + 'members.csv', format="csv", header="true")
# song_extra_info_df =  spark.read.load(raw_data_path + 'song_extra_info.csv', format="csv", header="true")
# songs_df = spark.read.load(raw_data_path + 'songs.csv', format="csv", header="true")
# test_df = spark.read.load(raw_data_path + 'test.csv', format="csv", header="true")
# train_df = spark.read.load(raw_data_path + 'train.csv', format="csv", header="true")

In [5]:
member_df.head()

Row(msno='XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=', city='1', bd='0', gender=None, registered_via='7', registration_init_time='20110820', expiration_date='20170920')

In [6]:
member_df.count()

34403

In [17]:
#preprocess
#reference: https://github.com/VasiliyRubtsov/wsdm_music_recommendations/blob/master/pipeline.ipynb
date_columns = ['expiration_date', 'registration_init_time']

train_data = pd.read_csv('music_data_raw/train.csv')
test_data = pd.read_csv('music_data_raw/test.csv', index_col=0)
item_data = pd.read_csv('music_data_raw/songs.csv')
user_data = pd.read_csv('music_data_raw/members.csv', parse_dates=date_columns)

In [18]:
all_data = pd.concat([train_data, test_data])

all_data = all_data.merge(item_data, on='song_id', how='left')
all_data = all_data.merge(user_data, on='msno', how='left')

/home/cs229/.local/lib/python3.5/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [9]:
all_data.head()

,msno,song_id,source_screen_name,source_system_tab,source_type,target,song_length,genre_ids,artist_name,composer,lyricist,language,city,bd,gender,registered_via,registration_init_time,expiration_date
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,Explore,explore,online-playlist,1.0,206471.0,359,Bastille,Dan Smith| Mark Crew,NaN,52.0,1,0,NaN,7,2012-01-02,2017-10-05
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,Local playlist more,my library,local-playlist,1.0,284584.0,1259,Various Artists,NaN,NaN,52.0,13,24,female,9,2011-05-25,2017-09-11
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,Local playlist more,my library,local-playlist,1.0,225396.0,1259,Nas,N. Jones、W. Adams、J. Lordan、D. Ingle,NaN,52.0,13,24,female,9,2011-05-25,2017-09-11
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,Local playlist more,my library,local-playlist,1.0,255512.0,1019,Soundway,Kwadwo Donkoh,NaN,-1.0,13,24,female,9,2011-05-25,2017-09-11
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,Explore,explore,online-playlist,1.0,187802.0,1011,Brett Young,Brett Young| Kelly Archer| Justin Ebach,NaN,52.0,1,0,NaN,7,2012-01-02,2017-10-05


In [20]:
enc = LabelEncoder()

for col in [
    'msno', 'song_id', 'source_screen_name', 
    'source_system_tab', 'source_type', 'genre_ids', 
    'artist_name', 'composer', 'lyricist', 'gender'
]:
    all_data[col] = enc.fit_transform(all_data[col].fillna('nan'))
    
for col in ['language', 'city', 'registered_via']:
    all_data[col] = enc.fit_transform(all_data[col].fillna(-2))
    

all_data['time'] = all_data.index / len(all_data)

n = len(train_data)
#train_data = all_data[:n]
#test_data = all_data[n:]

# train_data.to_hdf('music_data_raw/train_data.hdf', key='wsdm')
# test_data.to_hdf('music_data_raw/test_data.hdf', key='wsdm')

In [21]:
y = all_data['target']
X = all_data.drop(['target'], axis = 1)
train_X, test_X, train_y, test_y = train_test_split(X,y,test_size=0.4,random_state=1)
test_X, val_X, test_y, val_y = train_test_split(test_X,test_y,test_size=0.5,random_state=1)

In [12]:
# train_data.to_csv('music_data_raw/train_data_updated.csv')
# test_data.to_csv('music_data_raw/test_data_updated.csv')

In [22]:
train_X.to_csv('music_data_raw/train_X.csv')
test_X.to_csv('music_data_raw/test_X.csv')
val_X.to_csv('music_data_raw/valid_X.csv')
train_y.to_csv('music_data_raw/train_Y.csv')
test_y.to_csv('music_data_raw/test_Y.csv')
val_y.to_csv('music_data_raw/valid_Y.csv')

/home/cs229/.local/lib/python3.5/site-packages/ipykernel_launcher.py:4: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  after removing the cwd from sys.path.
/home/cs229/.local/lib/python3.5/site-packages/ipykernel_launcher.py:5: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """
/home/cs229/.local/lib/python3.5/site-packages/ipykernel_launcher.py:6: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  
